In [206]:
from openai import OpenAI
import pandas as pd
import IPython.display as ipd
import random
import numpy as np
import os
from collections import defaultdict
import random
from eedi.datasets import make_nice_df, make_complete_query
from pydantic import BaseModel
from dotenv import load_dotenv
import seaborn as sns
load_dotenv()

True

In [227]:
df_train = pd.read_csv("../data/train.csv")
df_train = make_nice_df(df_train)
# df_train["QuestionComplete"] = df_train.apply(make_complete_query, axis=1)
df_mis = pd.read_csv("../data/misconception_mapping.csv")
orig_mis = df_mis["MisconceptionName"]

In [ ]:
mis_in_train = set(df_train["MisconceptionId"])
all_mis = set(df_mis["MisconceptionId"])
missing = all_mis - mis_in_train
train_and_missing_ratio = len(mis_in_train) / len(missing)
len(missing), len(mis_in_train)

(983, 1604)

In [265]:
df_train.shape

(4370, 12)

4915

In [ ]:

train_and_missing_ratio

1.631739572736521

In [172]:
df_train["MisconceptionId"].value_counts()

MisconceptionId
1214    54
1379    43
2316    38
1507    36
1990    33
        ..
138      1
1140     1
1640     1
1862     1
95       1
Name: count, Length: 1604, dtype: int64

In [229]:
# using inverse sampling from train in order to balance which misconception to pick
mis_id_to_train_idx = defaultdict(list)
for train_idx, mis_id in enumerate(df_train["MisconceptionId"]):
    mis_id_to_train_idx[mis_id].append(train_idx)

mis_id_to_count = {k: len(v) for k, v in mis_id_to_train_idx.items()}
count_arr = np.array(list(mis_id_to_count.values()))
inv_prob = 1 / count_arr
inv_prob = inv_prob / inv_prob.sum()
aa = np.random.choice(
    np.array(list(mis_id_to_count.keys())), size=30000, replace=True, p=inv_prob
)

In [226]:
df_train

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectChoice,CorrectText,QuestionText,WrongChoice,WrongText,MisconceptionId,QuestionId_Answer
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\( 3 \times(2+4)-5 \),\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,D,Does not need brackets,1672,0_D
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",A,\( m+1 \),2142,1_A
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",B,\( m+2 \),143,1_B
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",C,\( m-1 \),2142,1_C
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Only\nKatie,Tom and Katie are discussing the \( 5 \) plant...,A,Only\nTom,1287,2_A
...,...,...,...,...,...,...,...,...,...,...,...,...
4365,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Only Katie,Tom and Katie are discussing congruence and si...,C,Both Tom and Katie,2312,1867_C
4366,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Only Katie,Tom and Katie are discussing congruence and si...,D,Neither is correct,2312,1867_D
4367,1868,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,B,Only Paul,Jo and Paul are arguing about how to fully des...,A,Only\nJo,801,1868_A
4368,1868,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,B,Only Paul,Jo and Paul are arguing about how to fully des...,C,Both Jo and Paul,801,1868_C


In [262]:
df_train["MisconceptionId"]

0       1672
1       2142
2        143
3       2142
4       1287
        ... 
4365    2312
4366    2312
4367     801
4368     801
4369      95
Name: MisconceptionId, Length: 4370, dtype: int64

In [263]:
row

QuestionId                                                         771
ConstructId                                                        665
ConstructName        Recognise graphical representations of inverse...
SubjectId                                                          244
SubjectName                              Indirect (Inverse) Proportion
CorrectChoice                                                        C
CorrectText          ![A curved line from top left rapidly descendi...
QuestionText         Which of the following graphs is most likely t...
WrongChoice                                                          B
WrongText            ![A straight line from top left to bottom righ...
MisconceptionId                                                     16
QuestionId_Answer                                                771_B
Name: 1816, dtype: object

In [250]:
M = aa[3]
print(M)  # that means we call openai for misconcption id M
row_idx = random.choice(mis_id_to_train_idx[M])
row = df_train.loc[row_idx]
row

system_prompt = """"""

prompt = f"""
# One-shot example:
subject: {row['SubjectName']}

construct: {row['ConstructName']}

question: {row['QuestionText']}

correct_answer: {row['CorrectText']}

wrong_answer: {row['WrongText']}

misconception: {orig_mis[M]}

# Create new questions based on given example
subject: {row['SubjectName']}

construct: {row['ConstructName']}

question: ...

correct_answer: ...

wrong_answer: ...
"""

print(prompt)

16

# One-shot example:
subject: Indirect (Inverse) Proportion

construct: Recognise graphical representations of inverse proportion in the form y = k/x

question: Which of the following graphs is most likely to illustrate the time taken to complete a job \( (t) \) changes depending on the number of people working on it \( (p) \)

correct_answer: ![A curved line from top left rapidly descending as it moves right before slowing down and levelling out ]()

wrong_answer: ![A straight line from top left to bottom right, starting from above the origin]()

misconception: Believes that graphs of inverse proportion meet the axes

# Create new questions based on given example
subject: Indirect (Inverse) Proportion

construct: Recognise graphical representations of inverse proportion in the form y = k/x

question: ...

correct_answer: ...

wrong_answer: ...



In [55]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
client

In [255]:
class Generated(BaseModel):
    question: str
    correct_answer: str
    wrong_answer: str

class GeneratedList(BaseModel):
    generated_list: list[Generated]

class Datapoint(BaseModel):
    subject: str
    construct_name: str  # can't use `construct` because it shadows pydantic internals
    question: str
    correct_answer: str
    wrong_answer: str

In [256]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are a mathematics teacher tasked to create questions to assess the student's understanding of math concepts. You will be presented with one example: the math question, the correct and wrong answer, along with the math misconceptions that led students choosing wrong answer instead of the correct one. Your task is to create similar, but diverse set of new questions, correct and wrong answers according to the given subject, construct, and misconception. Make sure the generated contents are diverse enough. Generate 5 set. Output answer in json.",
        },
        {"role": "user", "content": prompt},
    ],
    response_format=GeneratedList,
)

In [261]:
result = completion.choices[0].message.parsed
assert result is not None
for generated in result.generated_list:
    prompt = f"""
question: {generated.question}

correct_answer: {generated.correct_answer}

wrong_answer: {generated.wrong_answer}
"""
    print(prompt)
    print("🔥")


question: Which of the graphs below most accurately represents how the speed of a car \( s \) affects the travel time \( t \) for a fixed distance?

correct_answer: ![A curve starting high on the y-axis, dips sharply, and approaches the x-axis without touching it]

wrong_answer: ![A diagonal line slanting downwards from left to right intersecting both axes]

🔥

question: Select the graph that best depicts how the intensity of light \( I \) varies with the distance \( d \) from the source.

correct_answer: ![A hyperbolic curve decreasing steeply as it moves rightward, getting closer to the x-axis while never crossing it]

wrong_answer: ![A constant sloping line from top-left to bottom-right intersecting both axes]

🔥

question: Identify the graph which depicts the cooling rate \( R \) of a hot object decreasing as the ambient temperature gap \( T \) increases.

correct_answer: ![Graph exhibiting a steep drop quickly leveling out, not intersecting axes]

wrong_answer: ![Graph with a lin